# step 37. Tensor handling

In [1]:
# element-wise operation

import numpy as np
import dezero.functions as F
from dezero import Variable

x = Variable(np.array(1.0))
y = F.sin(x)
print(y)

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sin(x)
print(y)

x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
y = x + c
print(y)

variable(0.8414709848078965)
variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])
variable([[11 22 33]
          [44 55 66]])


### tensor 사용 시 역전파
지금까지의 역전파 구현은 '스칼라'를 대상으로 하였음. 텐서를 사용한 계산에 역전파를 적용하려면?<br/>
사실 이미 텐서에 대해서도 역전파 정상적으로 진행 될 것임.<br/>
* 그동안 스칼라를 대상으로 역전파를 구현하였음
* 지금까지 구현한 DeZero 함수에 '텐서'를 전달하면 텐서의 원소마다 '스칼라'로 계산한다.
* 텐서의 원소별 '스칼라'계산이 이루어지면 '스칼라'를 가정해 구현한 역전파는 '텐서'의 원소별 계산에서도 성립

In [3]:
# 원소별 연산에서는 역전파도 미분을 원소별로 곱하여 구한다.
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
t = 2*(x + c)
y = F.sum(t)

y.backward(retain_grad = True)
print(y.grad)
print(t.grad)
print(x.grad)
print(c.grad)

variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[2 2 2]
          [2 2 2]])
variable([[2 2 2]
          [2 2 2]])


기울기의 shape와 데이터의 shape가 일치한다는 것에 주목하자.  
즉, x.shape == x.grad.shape (다른변수들도 마찬가지)  
이 성질을 이용하여 원소별 계산이 아닌 함수 (sum, reshape를 구현할 수 있다.)

# step 38. 형상 변환 함수
앞으로의 step들에서는 원소별 계산이 아닌 다른 함수들에 대해서도 살펴보려고 한다.  
그것을 위해 이번 단계에서 reshape와 transpose 함수를 구현해야한다.

In [4]:
# numpy의 reshape
import numpy as np

x = np.array([[1,2,3],[4,5,6]])
y = np.reshape(x, (6,))
print(y)

[1 2 3 4 5 6]


reshape 함수는 구체적인 계산은 아무것도 하지 않고 단순히 형상(shape)만 변환한다.  
따라서 역전파 시 출력 쪽에서 전해지는 기울기에 어떠한 연산없이 그대로 입력쪽으로 흘려보내는데,  
이때 기울기의 형상이 입력의 형상과 같아지도록 변환한다.
<img src = "https://github.com/changdaeoh/DeepLearning-from-scratch/blob/main/images/38_1.png?raw=true">

In [ ]:
# reshape 구현
class Reshape(Function):
    def __init__(self, shape):
        self.shape = shape # 변형 목표가 되는 shape
    
    def forward(self, x):
        self.x_shape = x.shape # x의 original shape를 기억
        y = x.reshape(self.shape) # 넘파이의 reshape함수 이용
        return y
    
    # 연산 없이 형상만 변형 (원래 x의 shape대로)
    def backward(self, gy):
        return reshape(gy, self.x_shape) # gy는 Variable 객체이므로 별도의 reshape함수 만들어야함
    
from dezero.core import as_variable

def reshape(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return Reshape(shape)(x)

In [2]:
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.reshape(x, (6,))
y.backward(retain_grad = True)
print(y.grad)
print(x.grad)

variable([1 1 1 1 1 1])
variable([[1 1 1]
          [1 1 1]])


In [3]:
# numpy의 transpose
x = np.array([[1,2,3],[4,5,6]])
y = np.transpose(x)
print(y)

[[1 4]
 [2 5]
 [3 6]]


In [ ]:
# transpose 구현
class Transpose(Function):
    def forward(self, x):
        y = np.transpose(x)
        return y
    def backward(self, gy):
        gx = transpose(gy) # 출력쪽에서 전해지는 기울기를 전치
        return gx
    
def transpose(x):
    return Transpose()(x)

In [ ]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.transpose(x)
y.backward(retain_grad = True)
print(y.grad)
print(x.grad)

print(x.transpose())
print(x.T)

# step 39. sum function

덧셈의 순전파와 역전파
<img src = "https://github.com/changdaeoh/DeepLearning-from-scratch/blob/main/images/39_1.png?raw=true" width = "40%" height = "40%">
sum 함수의 순전파와 역전파
<img src = "https://github.com/changdaeoh/DeepLearning-from-scratch/blob/main/images/39_2.png?raw=true" width = "40%" height = "40%">

In [ ]:
# sum 함수 구현 - 입력변수의 형상과 같아지도록 기울기 원소를 복사
class Sum(Function):
    def forward(self, x):
        self.x_shape = x.shape # input의 원본 shape를 복사
        y = x.sum()
        return y
    
    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx
    
def sum(x):
    return Sum()(x)

In [1]:
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([[1,2,3,4,5,6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

variable(21)
variable([[1 1 1 1 1 1]])
variable(21)
variable([[1 1 1]
          [1 1 1]])


In [4]:
# axis와 keepdims
x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x, axis = 0) # 덧셈 수행 축 설정
print(y)
print(x.shape, ' -> ', y.shape)

x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x, keepdims = True) # 축의 개수 유지
print(y)
print(y.shape)

[5 7 9]
(2, 3)  ->  (3,)
[[21]]
(1, 1)


In [3]:
# dezero의 sum함수
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x, axis = 0) 
y.backward()
print(y)
print(x.grad)

variable([5 7 9])
variable([[1 1 1]
          [1 1 1]])


# step 40. broadcast

In [1]:
import numpy as np

x = np.array([1,2,3])
y = np.broadcast_to(x, (2,3))
print(y)

[[1 2 3]
 [1 2 3]]


broadcast 함수의 순전파 -> 지정한 shape가 되게 원소들을 복사  
broadcast 함수의 역전파 -> 복사된 기울기를 원래의 shape에 맞게 summation
<그림>

sum to 함수의 순전파 -> 배열의 원소들을 지정한 shape으로 summation
sum to 함수의 역전파 -> 원래의 shape에 맞게 기울기를 복사
<그림>

In [ ]:
# broadcast와 sum_to 함수는 상호 의존적
class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape
    
    def forward(self, x):
        self.x_shape = x.shape # save original shape 
        y = np.broadcast_to(x, self.shape)
        return y
    
    def backward(self, gy):
        gx = sum_to(gy, self.x_shape)
        return gx
    
def broadcast_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return BroadcastTo(shape)(x)


from dezero import untils

class SumTo(Function):
    def __init__(self, shape):
        self.shape = shape
        
    def forward(self, x):
        self.x_shape = x.shape # save original shape 
        y = utils.sum_to(x, self.shape)
        return y
    
    def backward(self, gy):
        gx = broadcast_to(gy, self.x_shape)
        return gx
    
def sum_to(x, shape):
    if x.shape == shape:
        return as_variable(x)
    return SumTo(shape)(x)

### 브로드캐스트 대응
broadcast는 형상이 다른 다차원 배열끼리의 연산을 가능하게함

In [2]:
x0 = np.array([1,2,3])
x1 = np.array([10])
y = x0 + x1
print(y)

[11 12 13]


In [1]:
# dezero의 Variable는 ndarray 인스턴스를 사용하여 구현하였기에 순전파시 자동 브로드캐스트
# 그러나 역전파는 제대로 이뤄지지 않음 -> ADD 클래스를 수정해줘야함
import numpy as np
from dezero import Variable

x0 = Variable(np.array([1,2,3]))
x1 = Variable(np.array([10]))
y = x0 + x1
print(y)

y.backward()
# 변수의 shape에 맞게 기울기가 출력!
print(x0.grad)
print(x1.grad)

variable([11 12 13])
variable([1 1 1])
variable([3])


# step 41. MatMul

In [2]:
# numpy에서의 내적, 행렬곱
a = np.array([1,2,3])
b = np.array([4,5,6])
c = np.dot(a,b)
print(c)

a = np.array([[1,2],[3,4]])
b = np.array([[5,6],[7,8]])
c = np.dot(a, b)
print(c)

32
[[19 22]
 [43 50]]


행렬곱 y = XW 에 대한 역전파
<그림41_6>

In [ ]:
class MatMul(Function):
    def forward(self, x, W):
        y = x.dot(W)
        return y
    
    def backward(self, gy):
        x, W = self.inputs
        gx = matmul(gy, W.T)
        gW = matmul(x.T, gy)
        return gx, gW

def matmul(x, W):
    return MatMul()(x, W)

In [1]:
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.random.randn(2,3))
W = Variable(np.random.randn(3,4))
y = F.matmul(x, W)
y.backward()

print(x.grad.shape)
print(W.grad.shape)

(2, 3)
(3, 4)
